# [LVV-T2730] - StarTracker Pointing and Tracking Test - Forward Azimuth Pattern

Collect data with the StarTracker following the azimuth pattern -270, -180, -90, 0, 90, 180, 270 deg.  
Nominal at four elevation angle 15, 45, 75, 86.5 deg.  
Minimum at the three angle: 15, 45, 86.5 deg.  

This test is forseen the first of four tests takes about one summer night (7 hours) in full version and a bit more than 5hours in the shortened version.  

Can we track for 10 min without moving the dome?

**Requirements:**
 - All the MT components should be enabled.
 - Need the `notebooks_vandv` module installed.

Please, see the [README] file for the requirements to run this notebook.  
The log messages printed in this notebook are stored in the EFD Script using the `-TTTTMMDD` format, where `TTTT` are the four last digits of the test case, `MM` is the 0-padded month, and `DD` is the 0-padded day of execution.

[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[LVV-T2730]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2730

## Setup

In [ ]:
test_case = "LVV-T2730"
test_exec = "LVV-EXXXX"

# az_grid = [-270, -180, -90, 0, 90, 180, 270]
az_grid = [180, 90, 0, -90]
# el_grid = [16, 45, 75, 86.5] # Nominal Elevation Angle - M1M3 goes to fault at 16 deg.
el_grid = [20, 45, 60, 75, 86] # Nominal Elevation Angle
# el_grid_min = [15, 45, 86.5] # Minimum Elevation Angle

mag_limit = 4
mag_range = 2
radius = 5.0

# 101 - Narrow Camera
# 102 - Wide Camera
# 103 - DIMM
camera_sal_indexes = [101, 102, 103]
exposure_times = [5., 3., 1.]  # s
base_msg = f"{test_case} {test_exec}:"

track_time = 30.  # 10 min
number_of_exposures = 2

# Set this to True to have the cameras taking images at the same time
sync_images = True

### Prepare Notebook

Start with importing libraries:

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import sys
import yaml

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.MTDome import SubSystemId
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control import RotType
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control.generic_camera import GenericCamera

from lsst.sitcom import vandv

---
Print out the execution info for future reference.

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

---
Use the `maintel/setup_mtcs.py` script in LOVE or the [LVV-T2344] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2344]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2344

---
Setup the logger, the Domain, and the remote for the telescope control.

In [ ]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)
log = logging.getLogger(f"{test_case}")

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

---
The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [ ]:
script = salobj.Controller("Script", index=199)
await script.start_task

In [ ]:
script.log.info("foo")

Instanciate the MTCS.

In [ ]:
mtcs = MTCS(domain=domain, log=script.log)
vandv.logger.add_filter_to_mtcs()
await mtcs.start_task

--- 
Initialize the GenericCamera and ComCam.  
The ComCam is used to ensure that we can take exposures using two cameras at the same time.  
You might need to modify the cell below. 

In [ ]:
camera_list = []

for index in camera_sal_indexes:

    gencam = GenericCamera(domain=domain, index=index, log=script.log)
    await gencam.start_task
    await gencam.enable()

    camera_list.append(gencam)

Run the cell below to hide all the messages related to the DDS and have a cleaner notebook.

In [ ]:
vandv.logger.add_filter_to_mtcs()

Default wait time in the steps.

In [ ]:
STDWAIT = 20

### Check CSC Status

In [ ]:
await mtcs.assert_liveliness()

In [ ]:
await mtcs.assert_all_enabled()

In [ ]:
for cam in camera_list:
    await cam.assert_liveliness()
    await cam.assert_all_enabled()

### Confirm take image each camera

In [ ]:
await camera_list[0].take_object(exptime=1., reason="sanity_check")

In [ ]:
await camera_list[1].take_object(exptime=1., reason="sanity_check")

In [ ]:
await camera_list[2].take_object(exptime=1., reason="sanity_check")

### Confirm Take Images in Sync

In [ ]:
res = (number_of_exposures) * len(camera_list)
for n in range(number_of_exposures):
    tasks = [asyncio.create_task(cam.take_object(exptime, reason=base_msg[:-1])) 
             for (cam, exptime) in zip(camera_list, exposure_times)]
    await asyncio.gather(*tasks)

### Other preparation

- LVV-T2713 (1.0) Establish TMA - StarTracker Axis Angle Reference

## Helper Functions

In [ ]:
def generate_azel_sequence(az_seq, el_seq, el_limit=90):
    """A generator that cicles through the input azimuth and elevation sequences
    forward and backwards.
    
    Parameters
    ----------
    az_seq : `list` [`float`]
        A sequence of azimuth values to cicle through
    el_seq : `list` [`float`]
        A sequence of elevation values to cicle through
    el_limit : `float`
        Cut off limit angle in elevation to skip points when going down. 
    Yields
    ------
    `list`
        Values from the sequence.
    Notes
    -----
    This generator is designed to generate sequence of values cicling through
    the input forward and backwards. It will also reverse the list when moving
    backwards.
    Use it as follows:
    >>> az_seq = [0, 180]
    >>> el_seq = [15, 45]
    >>> seq_gen = generate_azel_sequence(az_seq, el_seq)
    >>> next(seq_gen)
    [0, 15]
    >>> next(seq_gen)
    [0, 45]
    >>> next(seq_gen)
    [180, 45]
    >>> next(seq_gen)
    [180, 15]
    >>> next(seq_gen)
    [0, 15]
    """
    i = 1
    for az in az_seq:
        for el in el_seq[::i]:
            if el > el_limit and i == -1:
                continue
            else:
                yield (az, el)
        i *= -1

In [ ]:
async def take_images_for_time(cam, exptime, reason, tracktime):
    """ Takes images while tracking for some time. 
    
    Parameters
    ----------
    cam : `lsst.ts.observatory.control.base_camera.BaseCamera`
        Contains a camera instance. 
    exptime : `float`
        The exposure time.
    reason : `str`
        Reason passed to the `take_object` command.
    tracktime : `float`
        How long will we be tracking?
        
    Returns
    -------
    int : number of images obtained.
    """ 
    reason = reason.replace(" ", "_")
    timer_task = asyncio.create_task(asyncio.sleep(tracktime - exptime))
    n_images = 0

    while not timer_task.done():
        await cam.take_object(exptime, reason=reason)
        n_images += 1
        
    return n_images

In [ ]:
async def wait_for_dome_in_position():
    """Wait until the dome is in position"""
    await asyncio.sleep(20)
    azMotion = await mtcs.rem.mtdome.evt_azMotion.aget()

    while not azMotion.inPosition:
        azMotion = await mtcs.rem.mtdome.evt_azMotion.aget()
        await asyncio.sleep(5.)
        
    if azMotion.state == 1.:
        await mtcs.rem.mtdome.cmd_exitFault.set_start()

In [ ]:
await camera_list[0].take_object(5)
await camera_list[1].take_object(2)

## Data Acquisition

## Loop with TMA Tracking and Dome Following

In [ ]:
# for az, el in generate_azel_sequence(az_grid, el_grid):
    
#     ## All of the following steps should be included in the point_azel command
#     ## - Point the Dome
#     ## - Wait the Dome to arrive    
#     ## - Point the TMA
#     ## - Wait the TMA to arrive
#     script.log.info(f"{base_msg} Data acquisition at az={az} and el={el} - Start")
    
#     # Enable error in the Dome for now
#     mtcs.check.mtdome = False
    
#     ## Disable dome following for now
#     script.log.info(f"{base_msg} Dome following - Enable")
#     await mtcs.enable_dome_following()    
    
#     ## Point Az/El
#     script.log.info(f"{base_msg} Point to az={az} (tma_az={tma_az}), el={el} - Start")
#     # radec_icrs = mtcs.radec_from_azel(az=az, el=el)
#     # await mtcs.slew_icrs(radec_icrs.ra, radec_icrs.dec)
#     await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=az, elevation=el)
#     script.log.info(f"{base_msg} Point to az={az} (tma_az={tma_az}), el={el} - Done")
    
#     ## Disable dome following
#     script.log.info(f"{base_msg} Dome following - Disable")
#     await mtcs.disable_dome_following()
    
#     exp_start = Time.now()
#     script.log.info(f"{base_msg} Take image(s) while tracking - Start")
            
#     tasks = [asyncio.create_task(take_images_for_time(cam, exptime, base_msg[:-1], track_time)) 
#              for (cam, exptime) in zip(camera_list, exposure_times)]
#     res = await asyncio.gather(*tasks)
    
#     script.log.info(f"{base_msg} Take image(s) while tracking - Done")

#     exp_end = Time.now()
#     delta = exp_end - exp_start
#     delta.format = "sec"
#     script.log.info(f"{base_msg} Took {'/'.join([str(i) for i in res])} images in {delta.to_value(1 * u.s):.2f} seconds.")
    
#     script.log.info(f"{base_msg} Data acquisition at az={az} and el={el} - Done")

## Loop without TMA tracking and without Dome Following

The following cell is isolated to allow continuing the grid from a point where you stopped. 

In [ ]:
azel = [(az, el) for az, el in generate_azel_sequence(az_grid, el_grid)]

for i, (az, el) in enumerate(azel):
    print(i, az, el)

---
Uncomment and edit the lines below if you need a smaller grid for now.

In [ ]:
# az_grid_tiny = [130]
# el_grid_tiny = [40]
# azel = [(az, el) for az, el in generate_azel_sequence(az_grid_tiny, el_grid_tiny)]

---
Uncomment the line bellow and replace i with the position index from the loop below

In [ ]:
# azel = azel[i::]

---
This is the main loop. See comments in the code for details.

In [ ]:
## Ignore (set check to False) the following components:
mtcs.check.mtaos = False
mtcs.check.mtdome = False
mtcs.check.mtdometrajectory = False
mtcs.check.mthexapod_1 = False
mtcs.check.mthexapod_2 = False
mtcs.check.mtm1m3 = False
mtcs.check.mtm2 = False
mtcs.check.mtrotator = False

for i, (az, el) in enumerate(azel):
    
    ## All of the following steps should be included in the point_azel command
    ## - Point the Dome
    ## - Wait the Dome to arrive    
    ## - Point the TMA
    ## - Wait the TMA to arrive
    script.log.info(f"{base_msg} Data acquisition #{i} at az={az} and el={el} - Start")
    
    ## Disable dome following for now
    script.log.info(f"{base_msg} Dome following - Disable")
    # await mtcs.disable_dome_following()    
    mtcs.check.mtdometrajectory = False
    
    ## Start moving the Dome
    script.log.info(f"{base_msg} Moving Dome to az={az} - START")
    await mtcs.rem.mtdome.cmd_exitFault.set_start()
    await asyncio.sleep(5)
    await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az, velocity=0)
    dome_task = asyncio.create_task(wait_for_dome_in_position())
    
    ## Point Az/El
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Start")
    # await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=tma_az, elevation=el)
    radec = mtcs.radec_from_azel(az, el)
    await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Done")
    
    ## Wait until dome in position
    await dome_task
    script.log.info(f"{base_msg} Moving Dome to az={az} - DONE")

    ## Sleep and engage dome breaks
    script.log.info(f"{base_msg} Dome Brakes - Start")
    await asyncio.sleep(10)
    sub_system_ids = SubSystemId.AMCS
    await mtcs.rem.mtdome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)
    script.log.info(f"{base_msg} Dome Brakes - Done")
    
    exp_start = Time.now()
    script.log.info(f"{base_msg} Take image(s) - Start")
    
    ## Take images out of sync for while tracking for some time.
    # tasks = [asyncio.create_task(take_images_for_time(cam, exptime, base_msg[:-1], track_time)) 
    #          for (cam, exptime) in zip(camera_list, exposure_times)]
    # res = await asyncio.gather(*tasks)
    
    ## Take images in sync - this will increase overhead on the camera with shorter exposure time.
    res = (number_of_exposures) * len(camera_list)
    for n in range(number_of_exposures):
        tasks = [asyncio.create_task(cam.take_object(exptime, reason=base_msg[:-1])) 
                 for (cam, exptime) in zip(camera_list, exposure_times)]
        await asyncio.gather(*tasks)

    script.log.info(f"{base_msg} Take image(s) - Done")

    exp_end = Time.now()
    delta = exp_end - exp_start
    delta.format = "sec"
    script.log.info(f"{base_msg} Took {'/'.join([str(i) for i in res])} images in {delta.to_value(1 * u.s):.2f} seconds.")
    
    script.log.info(f"{base_msg} Data acquisition #{i} at az={az} and el={el} - Done")
    await mtcs.rem.mtdome.cmd_exitFault.set_start()

## Slew and track a single target using Ra/Dec

Read Ra/Dec and convert to Az/El:

In [ ]:
coord = SkyCoord("05h23m27.4s -69°43'59.5", unit=(u.hourangle, u.deg), frame="icrs")
print(coord)

azel = mtcs.azel_from_radec(coord.ra, coord.dec)
az, el = azel.az.deg, azel.alt.deg
print(az, el)

In [ ]:
## Disable dome following for now
script.log.info(f"{base_msg} Dome following - Disable")
# await mtcs.disable_dome_following()    
mtcs.check.mtdometrajectory = False

## Start moving the Dome
script.log.info(f"{base_msg} Moving Dome to az={az} - START")
await mtcs.rem.mtdome.cmd_exitFault.set_start()
await asyncio.sleep(5)
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az, velocity=0)
dome_task = asyncio.create_task(wait_for_dome_in_position())

## Point Az/El
script.log.info(f"{base_msg} Point to az={az}, el={el} - Start")
# await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=tma_az, elevation=el)
radec = mtcs.radec_from_azel(az, el)
await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)
script.log.info(f"{base_msg} Point to az={az}, el={el} - Done")

## Wait until dome in position
await dome_task
script.log.info(f"{base_msg} Moving Dome to az={az} - DONE")

## Sleep and engage dome breaks
script.log.info(f"{base_msg} Dome Brakes - Start")
await asyncio.sleep(10)
sub_system_ids = SubSystemId.AMCS
await mtcs.rem.mtdome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)
script.log.info(f"{base_msg} Dome Brakes - Done")

In [ ]:
await mtcs.offset_azel(az=0, el=3600*0.5)

## Dome Commands

The following commands allow to control the Dome. However, make sure it is in ENABLED state. Use LOVE to change its state.

### Move to position

In [ ]:
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=186, velocity=0)

### Stop the Dome

In [ ]:
sub_system_ids = SubSystemId.AMCS
await dome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)

### Recover from fault when stopping

In [ ]:
await mtcs.rem.mtdome.cmd_exitFault.set_start()